<a href="https://colab.research.google.com/github/Shirouky/map/blob/main/Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git init
!git config --global user.email "mephiderebas@gmail.com"
!git config --global user.name "Lubov Derebas"

Reinitialized existing Git repository in /content/.git/


# Импорт

In [ ]:
pip install osmnx

In [ ]:
from __future__ import annotations
from abc import ABC, abstractmethod
from datetime import datetime
import random
import matplotlib.pyplot as plt
import folium
import osmnx as ox
import networkx as nx
import pandas as pd
import random
import json
from datetime import datetime, timedelta
import numpy

# Factory

In [ ]:
def random_time_between(start_time, end_time):
    start_timestamp = start_time.timestamp()
    end_timestamp = end_time.timestamp()
    random_timestamp = random.uniform(start_timestamp, end_timestamp)
    return datetime.fromtimestamp(random_timestamp)

In [ ]:
def time(hour):
  return datetime.strptime(f"{hour}:00 20-04-2025", "%H:%M:%S %d-%m-%Y")

In [ ]:
class Factory(ABC):
    @abstractmethod
    def factory_method(self):
        pass

    def create(self) -> str:
        person = self.factory_method()

        return person

In [ ]:
class DoctorFactory(Factory):
    def factory_method(self) -> Person:
      name = "doctor"
      places = [["amenity", "hospital", "time", random_time_between(time("6:00"), time("8:30"))],
                  ["shop", True, "time", random_time_between(time("17:00"), time("19:30"))],
                  ["building", ["apartments", "residential", "house"], "delta", timedelta(minutes=random.randint(5, 40))]]

      return Person(name, places)

In [ ]:
class SchoolChildFactory(Factory):
    def factory_method(self) -> Person:
        name = "schoolchild"
        places = [["building", "school", "time", random_time_between(time("7:00"), time("8:30"))],
                  ["shop", True, "time", random_time_between(time("14:00"), time("16:30"))],
                  ["building", ["apartments", "residential", "house"], "delta", timedelta(minutes=random.randint(5, 20))]]

        return Person(name, places)

In [ ]:
class OfficeWorkerFactory(Factory):
    def factory_method(self) -> Person:
      name = "office worker"
      places = [["office", "company", "time", random_time_between(time("7:00"), time("8:30"))],
                  ["leisure", "sports_centre", "time", random_time_between(time("17:00"), time("19:00"))],
                  ["building", ["apartments", "residential", "house"], "delta", timedelta(minutes=random.randint(40, 180))]]

      return Person(name, places)

In [ ]:
class CourierFactory(Factory):
    def factory_method(self) -> Person:
      name = "courier"
      places = [["office", "logistics", "time", random_time_between(time("7:30"), time("9:30"))],
                  ["building", ["apartments", "residential", "house"], "delta", timedelta(minutes=random.randint(5, 15))],
                  ["amenity", "fast_food", "delta", timedelta(minutes=random.randint(5, 15))],
                  ["building", "residential", "delta", timedelta(minutes=random.randint(15, 45))],
                  ["office", "logistics", "delta", timedelta(minutes=random.randint(5, 15))],
                  ["building", ["apartments", "residential", "house"], "delta", timedelta(minutes=random.randint(20, 30))]]

      return Person(name, places)

In [ ]:
class SalesmanFactory(Factory):
    def factory_method(self) -> Person:
      name = "salesman"
      places = [["shop", True, "time", random_time_between(time("8:00"), time("9:30"))],
                  ["amenity", "pharmacy", "time", random_time_between(time("18:00"), time("19:00"))],
                  ["building", ["apartments", "residential", "house"], "delta", timedelta(minutes=random.randint(5, 20))]]

      return Person(name, places)

In [ ]:
class Person:
    places = []
    name = ""

    def __init__(self, name: str, places: list):
        self.name = name
        self.places = places

    def get_places(self):
      return self.places

    def get_name(self):
      return self.name

# Визуализация

In [ ]:
def visualize_png(graph, routes, place, uid):
    route = []
    for r in routes:
      route += r
    ox.plot_graph_route(
        graph,
        route,
        route_color='red',
        route_linewidth=4,
        node_size=0,
        bgcolor='white',
        show=False,
        close=False
    )
    plt.title(f"Маршрут перемещения в {place} человека с uid {uid}")
    plt.savefig("route_plot.png")
    plt.show()

In [ ]:
def visualize_interactive(G, routes, name, uid):
    # Интерактивная карта с folium (открывается в браузере)
    print("\nСоздание интерактивной карты (folium)...")
    # Создаем карту с центром в первой точке маршрута
    m = folium.Map(
          location=(G.nodes[routes[0][0]]['y'], G.nodes[routes[0][0]]['x']),
          zoom_start=14,
          tiles="OpenStreetMap"
      )

    for route in routes:
      # Получаем координаты всех точек маршрута
      route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]

      # Добавляем линию маршрута
      folium.PolyLine(
          route_coords,
          color="red",
          weight=5,
          opacity=0.8
      ).add_to(m)

      # Добавляем маркеры
      folium.Marker(
          route_coords[0],
          icon=folium.Icon(color="green")
      ).add_to(m)

    # Сохраняем HTML с картой
    m.save(f"interactive_route_{name}_{uid}.html")
    print("Интерактивная карта сохранена в interactive_route.html")


# Main

In [ ]:
def create_graph(place, network):
    graph = ox.graph_from_place(place, network_type=network)
    G = ox.convert.to_digraph(graph)

    return graph, G

In [ ]:
def json_save(output_data):
    with open('movement.json', 'w') as f:
        json.dump(output_data, f, indent=2)

    print("Данные о перемещении сохранены в movement.json")


In [ ]:
def calculate_time(start_point, end_point, has_car=True, is_raining=False, is_urgent=False):

    speeds = {
        'walk': 5000,
        'bike': 15000,
        'drive': 60000
    }

    if is_raining: speeds['walk'] *= 0.7
    if is_urgent: speeds['drive'] *= 1.2

    graphs = {}
    if has_car: graphs['drive'] = G_drive
    graphs.update({'walk': G_walk, 'bike': G_bike})

    best_time = float('inf')
    best_result = None

    for transport, G in graphs.items():
        try:
            route = nx.shortest_path(G, ox.nearest_nodes(G, *start_point[::-1]),
                                     ox.nearest_nodes(G, *end_point[::-1]),
                                     weight='length')
            dist = sum(ox.utils_graph.get_route_edge_attributes(G, route, 'length'))
            time = dist / speeds[transport] * 60  # В минутах

            if transport == 'drive': time += 5  # Парковка
            elif transport == 'bike': time += 3  # Замок

            if time < best_time:
                best_time = time
                best_result = (G, route, transport, timedelta(minutes=time))
        except:
            continue

    return best_result

In [ ]:
def generate_track(nodes, place, places):
    # Получаем случайный жилой дом
    gdf = ox.features.features_from_place(place, tags={"building": ["apartments", "residential", "house"]})
    houses = gdf[pd.notnull(gdf["building"])]
    home = houses.sample(1).iloc[0]
    home_point = home.geometry.centroid

    # Находим ближайший узел графа для стартовой точки
    start_node = ox.distance.nearest_nodes(G, home_point.x, home_point.y)
    start_address = f"Лобня, {home.get('addr:street', 'улица не указана')}, {home.get('addr:housenumber', 'номер не указан')}"

    movement_data = []
    current_node = start_node
    routes = []
    current_time = 0
    distance = 3000

    speeds = {
        'walk': 5000,
        'bike': 15000,
        'drive': 50000
    }
    transport = 'drive'

    for i, place_info in enumerate(places):
        tag, value, time_type, time_value = place_info

        if time_type == "time":
          current_time = time_value
        elif time_type == "delta":
          current_time += time_value

        print(current_time, tag)

        # Ищем объекты нужного типа в радиусе search_radius
        if i == len(places) - 1:  # Возвращаемся домой
            end_place = home
        else:
            # Поиск в радиусе от текущей позиции
            point = G.nodes[current_node]
            current_location = (point['y'], point['x'])

            try:
              nearby_objects = ox.features_from_point(
                  center_point=current_location,
                  dist=distance,
                  tags={tag: value}
              )
            except ox._errors.InsufficientResponseError as e:
              print(f"Не найдено объектов {tag}={value} в радиусе {distance} м")
              continue

            end_place = nearby_objects.sample(1).iloc[0]

        end_point = end_place.geometry.centroid
        end_node = ox.distance.nearest_nodes(G, end_point.x, end_point.y)

        # Находим ближайший узел графа для конечной точки
        end_node = ox.distance.nearest_nodes(G, end_point.x, end_point.y)
        # Строим маршрут
        try:
            new_route = nx.shortest_path(G, current_node, end_node, weight='length')
            routes.append(new_route)

            # Генерируем данные движения
            _point = {'y': G.nodes[current_node]['y'], 'x': G.nodes[current_node]['x']}
            for i, node in enumerate(new_route):
              point = G.nodes[node]
              lat, lon = point['y'], point['x']

              dist = ox.distance.euclidean(_point['y'], _point['x'], point['y'], point['x']) * 10 ** 5
              time_spent = round(dist / speeds[transport] * 3600)
              current_time += timedelta(seconds=time_spent)

              movement_entry = {
                  "latitude": int(lat * 10 ** 6),
                  "longitude": int(lon * 10 ** 6),
                  "timestamp": int(current_time.timestamp())
              }
              movement_data.append(movement_entry)
              _point = point

              current_node = end_node

        except nx.NetworkXNoPath:
            continue

    return routes, movement_data


In [ ]:
place = "Россия, Лобня"
female = 1054
male = 1000
rosstat_gender = [male / (female + male), female / (female + male)]

network = 'drive'
graph, G = create_graph(place, network)
nodes = list(G.nodes())
start_time = datetime.now()
output_data = []

In [ ]:
factories = [DoctorFactory(), SchoolChildFactory(), OfficeWorkerFactory(), SalesmanFactory()]

In [ ]:
for uid in range(10):
    person = random.choice(factories).create()
    print(person.get_name())
    routes, movement_data = generate_track(nodes, place, person.get_places())
    output_data.append({
        "id": uid,
        "track": movement_data,
        "optional_data": {
            "gender": int(numpy.random.choice((0, 1), p=rosstat_gender)),
            "age": random.randint(15, 50),
        },
    })
    visualize_interactive(G, routes, person.get_name(), uid)
    # visualize_png(graph, routes, place, uid)

json_save(output_data)

schoolchild
2025-04-20 07:20:55.800100 building
2025-04-20 15:56:13.382573 shop
2025-04-20 16:09:50.382573 building

Создание интерактивной карты (folium)...
Интерактивная карта сохранена в interactive_route.html
schoolchild
2025-04-20 07:46:23.628872 building
2025-04-20 16:14:11.528179 shop
2025-04-20 16:32:39.528179 building

Создание интерактивной карты (folium)...
Интерактивная карта сохранена в interactive_route.html
salesman
2025-04-20 09:26:15.350525 shop
2025-04-20 18:43:23.624307 amenity
2025-04-20 18:57:52.624307 building

Создание интерактивной карты (folium)...
Интерактивная карта сохранена в interactive_route.html
doctor
2025-04-20 06:35:22.422795 amenity
2025-04-20 17:54:46.387862 shop
2025-04-20 18:15:27.387862 building

Создание интерактивной карты (folium)...
Интерактивная карта сохранена в interactive_route.html
salesman
2025-04-20 09:04:47.406075 shop
2025-04-20 18:43:12.591923 amenity
2025-04-20 18:50:35.591923 building

Создание интерактивной карты (folium)...
Инте